<img src="https://raw.githubusercontent.com/sbarja/smart-energy-22-23/main/Figures/top_ML_smart.png" alt="Drawing" style="width: 1100px;"/>

# EJERCICIO
# Aprendizaje supervisado: Clasificación.

## *Clasificación binaria de precios de electricidad en el Mercado Diario*

**Objetivo:** Imaginando que estamos a medioados de 2020, predecir en qué horas el precio de la electricidad en el Mercado Diario será elevado, siendo la **clase 0** para valores menores a 40 €, y **clase 1** para valores mayores a 40 €.  Se utilizará el contexto y datos históricos del **2020** de la variable target que queremos clasificar y de otros atributos (features) que pueden ayudar a predecir modelo.


Una técnica ampliamente adoptada para tratar conjuntos de datos muy desequilibrados se denomina remuestreo. Consiste en eliminar muestras de la clase mayoritaria (submuestreo) y/o añadir más ejemplos de la clase minoritaria (sobremuestreo).

<img src="https://raw.githubusercontent.com/sbarja/smart-energy-22-23/main/Figures/ejercicio-clasificacion.png" alt="Drawing" style="width: 800px;"/>



### Antes de empezar:

* En el archivo **S4-data-precios.xlsx** se encuentra el conjunto de datos de entrada de este ejemplo (atributos + etiqueta). 
* Datos del 2 de enero 2020 al 26 de junio de 2020.


# Pasos para crear un modelo de machine learning

<img src="Figures/Fases.png" alt="Drawing" style="width: 800px;"/>

## **1. Importar librerías y datos**


In [ ]:
# Importamos las librerías
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Cargamos el conjunto de datos de entrada
dataset=pd.read_excel('Data\S4-data-precios.xlsx')

## **2. Comprender los datos**

Es necesario visualizar y comprender los datos con los que vamos a trabajar, así como conocer sus características. 

1. ¿Cuántos datos hay? ¿Cuántos atributos hay en los datos?  
2. ¿Qué significan?
3. ¿Falta algún dato?
4. ¿Están balanceadas las etiquetas? 
4. Resumen estadístico del conjunto de datos de entrada.

<div class="alert">
    <b> ¿Cuántos datos hay?¿Cuántos atributos hay en los datos? </b>
</div>


In [ ]:
# Filasxcolumnas de los datos
dataset.shape


In [ ]:
# Observa las primeras 5 filas de los datos
dataset.head(50)


<div class="alert alert-success">
    <b> ¿Qué significan? </b>
</div>

* ***[Hora, Día, Mes]*** Hora, día y mes de cada una de las observaciones. Son valores enteros *int64*.

* ***[Hidraul, Eolica, Ciclocomb, Cogener, Nuclear, Carbon, Biomas]*** se refiere a la energía programada horaria del programa PVP en el mercado diario por tipo de producción del día anterior.  Son valores reales *float*.

* ***[Demanda]*** es la totalidad de energía programada en el mercado diario eléctrico en España el día anterior.  Son valores reales *float*.

* ***[precio-elect-dia-anterior]*** precio de la electricidad el día anterior. Son valores reales *float*.

* ***[MIBGAS-dia-anterior]*** precio del gas natural el día anterior. Son valores reales *float*.

* ***[Clases]*** son las etiquetas de precio que queremos predecir. Son valores enteros *int64*.




In [ ]:
# Formato de los datos
dataset.dtypes


<div class="alert alert-success">
    <b> La etiqueta es de tipo ´´object´´, por lo que hay que transformarla a numérico </b>
</div>

No podemos ver la correlación con el precio, debemos pasarlo a numérico [LabelEncoder]


[LabelEncoder]: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html

In [ ]:
from sklearn.preprocessing import LabelEncoder

lab_encoder = LabelEncoder() 
dataset['precio'] = lab_encoder.fit_transform(dataset['precio'])
print(dataset)

In [ ]:
# replace column values 0: menor que 40; 1: mayor que 40. 

dataset['precio'] = dataset['precio'].map({0:1, 1:0})
dataset.to_excel('dataset.xlsx')


In [ ]:
dataset

<div class="alert alert-success">
    <b> ¿Falta algún dato? De ser así, indica cuántos y en que atributo </b>
</div>




In [ ]:
# Comprobar si falta algún dato y en qué atributo
dataset.isna().sum()

<div class="alert alert-success">
    <b> ¿Están balanceadas las etiquetas? </b>
</div>


In [ ]:
# Comprobar si las etiquetas están desvalanceadas
balance_clases = dataset['precio'].value_counts()
print(balance_clases)

# Gráfico del balance de clases
balance_clases.plot.pie()

<div class="alert alert-success">
    <b> Resumen estadístico del conjunto de datos de entrada: </b>
</div>
La estadística descriptiva recolecta y analiza el conjunto de datos de entrada con el objetivo de describir las características y comportamientos de este conjunto mediante las siguientes medidas resumen: número total de observaciones (count), media (mean), desviación estándar (std), valor mínimo (min), valor máximo (max) y los valores de los diferentes cuartiles (25%, 50%, 75%).

In [ ]:
# Datos estadísticos de cada uno de los atributos
dataset.describe()

## **3. Visualizar los datos**

Una manera visual de entender los datos de entrada. 
1. Histograma
2. Curva de densidad
3. Boxplots
4. Matriz de correlación


<div class="alert alert-success">
    <b>Histograma </b>
</div>


Respresentación gráfica de cada uno de los atributos en forma de barras, donde la superficie de la barra es proporcional a la frecuencia de los valores representados.

In [ ]:
histograma = dataset.hist(xlabelsize=10, ylabelsize=10, bins=50, figsize=(15, 10))

<div class="alert alert-success">
    <b> Gráfico de densidades </b>
</div>

Visualiza la distribución de los datos. Es una variable del histograma, pero elimina el ruido, por lo que son mejores para determinar la forma de distribución de un atributo. Lo spicos del gráfico de densidad ayudan a mostrar dónde los valores se concentran más. 

In [ ]:
density = dataset.plot(kind='kde', x=4, subplots=True, legend=True, layout=(4, 4), figsize=(17, 12), sharex=False,
                        fontsize=8, stacked=True) 

<div class="alert alert-success">
    <b> Boxplots </b>
</div>


El boxplot (diagrama de caja) nos permite identificar los valores atípicos y comparar distribuciones. Además, se conoce como se distribuyen el 50% de los valores (dentro de la caja).


In [ ]:
atributos_boxplot = dataset.plot(kind='box', subplots=True, layout=(4, 4), figsize=(15, 10), sharex=False,
                                 sharey=False, fontsize=10)

<div class="alert alert-success">
    <b> Matriz de correlación </b>
</div>

Utilizamos el método de Spearman para evaluar la relación monótona entre dos variables contínuas. 

Comparación entre método de [Pearson y Spearman]

[Pearson y Spearman]: https://support.minitab.com/es-mx/minitab/18/help-and-how-to/statistics/basic-statistics/supporting-topics/correlation-and-covariance/a-comparison-of-the-pearson-and-spearman-correlation-methods/


* **¿Qué variable no tiene ninguna correlación con ningún atributo?** 

In [ ]:
# Otra librería de visualización de datos
import seaborn as sns

# Cálculo de coeficientes de correlación
corr_matrix = dataset.corr(method='spearman') 


# Quitar valores repetidos
mask = np.zeros_like(corr_matrix)
mask[np.triu_indices_from(mask)] = True

  
f, ax = plt.subplots(figsize=(12, 12))
#Generar Heat Map,
sns.heatmap(corr_matrix, annot=True, fmt=".2f" , mask=mask,)
    # xticks
plt.xticks(range(len(corr_matrix.columns)), corr_matrix.columns);
    # yticks
plt.yticks(range(len(corr_matrix.columns)), corr_matrix.columns)
    # plot
plt.show()

## *4. Preparar los datos*

1. Missing data
2. Data cleaning (eliminar outliers).
3. LabelEncoding (ya lo hemos hecho)
4. Feature engineering
5. Transformación.

Primero, divido los datos en **atributos**: X (features) y **etiquetas**: y (target)

In [ ]:
# Atributos X (features); etiquetas y (target)
X = dataset.drop(['precio'], axis=1) 
y = dataset['precio']
X


<div class="alert alert-success">
    <b> Missing data </b>
</div>


Comprobar si exisiten Nan en los datos de entrada. 

- Se utiliza el método [fillna] de Pandas.

- Más información acerca de cómo imputar valores con [Scikit Learn]

[Scikit Learn]: https://scikit-learn.org/stable/modules/impute.html
[fillna]: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html





In [ ]:
# Comprobar si faltan datos en los atributos
X.isna().sum()

In [ ]:
# Relleno los missing values de cada atributo con el valor anterior del atributo. 
X["demanda"].fillna(method='ffill', inplace=True)
X["carbon"].fillna(method='bfill', inplace=True)

In [ ]:
# Comprobar si faltan datos en el target
y.isna().sum()

In [ ]:
# Comprueba que no falta ningún valor
X.isna().sum()


<div class="alert alert-success">
    <b> Feature engineering</b>
</div>



Utilizando la matriz de correlación, eliminar los atributos con una correlacion cercana a 0 con la etiqueta **"precio"**. 

* **¿Qué atributo(s) se elimana(n)?** 

In [ ]:
# Elimino el atributo
X.drop(['biomas'], axis='columns', inplace=True)
# dia lo eliminaremos más adelante

X

## *5. Dividir los datos*
 

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.20  # porcentaje de los datos de entrada que utilizaré para validar el modelo

# Divido los datos en datos de entreno, validación y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size,
                                                    shuffle=False)



In [ ]:
X_test

In [ ]:
y_train = pd.DataFrame(y_train)
y_train

### Resampling 

Una técnica ampliamente adoptada para tratar conjuntos de datos muy desequilibrados se denomina remuestreo. Consiste en eliminar muestras de la clase mayoritaria (submuestreo) y/o añadir más ejemplos de la clase minoritaria (sobremuestreo). **APLICAR SOLO EN EL TRAINING DATASET**

<img src="https://github.com/sbarja/curso-intro-machine-learning-2023/blob/main/figuras/resampling.png?raw=1" alt="Drawing" style="width: 800px;"/>

<div class="alert alert-success">
    <b> Oversampling de la clase minoritaria </b>
</div>


El sobremuestreo puede definirse como añadir más copias de la clase minoritaria. El sobremuestreo puede ser una buena opción cuando no se tiene muchos datos con los que trabajar.

Utilizaremos el módulo de remuestreo de Scikit-Learn para replicar aleatoriamente muestras de la clase minoritaria.

In [ ]:
# Creating a datetime column para ordenar el upsampling data
X_train['year'] = 2020  # Modify this year as needed for your dataset
# Ensure that the month, day, and year columns are integers if they aren't already
X_train['date'] = pd.to_datetime(X_train[['year', 'mes', 'dia']].astype(int).rename(columns={'mes': 'month', 'dia': 'day'}))
# If the 'hora' column should be included as the hour part of the datetime:
X_train['date'] += pd.to_timedelta(X_train['hora'], unit='h')

X_train

In [ ]:
y_train['date'] = X_train['date']
y_train.head(15)

In [ ]:
y_train.value_counts("precio")

In [ ]:
# Separar clases mayoritarias y minoritarias
X_train_majority = X_train[y_train['precio'] == 0]
X_train_minority = X_train[y_train['precio'] == 1]
y_train_majority = y_train[y_train['precio'] == 0]
y_train_minority = y_train[y_train['precio'] == 1]


In [ ]:
X_train_minority

In [ ]:
from sklearn.utils import resample

# upsample minority
X_train_minority_upsampled, y_train_minority_upsampled = resample(X_train_minority,y_train_minority,
                          replace=True, # sample with replacement
                          n_samples=len(y_train_majority), # match number in minority class
                          random_state=27) # Set random seed for reproducibility

# Combine the upsampled minority class with the majority class
X_train_upsampled = pd.concat([X_train_majority, X_train_minority_upsampled])
y_train_upsampled = pd.concat([y_train_majority, y_train_minority_upsampled])


In [ ]:
# Sort the combined DataFrame by the 'date' column
X_train_upsampled_sorted = X_train_upsampled.sort_values(by='date')
y_train_upsampled_sorted = y_train_upsampled.sort_values(by='date')

In [ ]:
X_train_upsampled_sorted.head(20)

In [ ]:
y_train_upsampled_sorted

<div class="alert alert-success">
    <b> Grafico de nuevo el balance de clases, para comprobar que efectivamente están balanceadas. </b>
</div>


In [ ]:
# Comprobar si las etiquetas están desvalanceadas
balance_clases = y_train_upsampled_sorted['precio'].value_counts()
print(balance_clases)

# Gráfico del balance de clases
balance_clases.plot.pie()

In [ ]:
# Eliminamos las columnas que no necesitamos
y_train_upsampled_sorted.drop(['date'], axis='columns', inplace=True)
X_train_upsampled_sorted.drop(['date', 'year'], axis='columns', inplace=True)



<div class="alert alert-success">
    <b> Transformación (escalado) </b>
</div>

* **Escalar los datos utilizando el método de *MinMaxScaler()* dentro del rango [0,1] o StandardScaler()**

In [ ]:
X_train_upsampled_sorted.head(5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Escalar los datos de entreno
scaler = StandardScaler()
X_train_scaled = X_train_upsampled_sorted.copy()
X_train_upsampled_scaled = pd.DataFrame(scaler.fit_transform(X_train_scaled))
X_train_upsampled_scaled.columns = X_train_upsampled_sorted.columns
X_train_upsampled_scaled.head()



In [ ]:
# Mostrar la media y desviación estándar del dataset escalado
print("Mean of standardized dataset:", scaler.mean_)
print("Standard deviation of standardized dataset:", scaler.scale_)

<div class="alert alert-success">
    <b> Escalamos los datos de TEST con la media y desviación estandar del dataset de entreno (X_train) </b>
</div>


In [ ]:

# Escalar los datos de test
X_test_scaled = X_test.copy()
X_test_scaled = pd.DataFrame(scaler.transform(X_test_scaled))
X_test_scaled.columns = X_test.columns
X_test_scaled.head()


<div class="alert alert-success">
    <b> Dividimos los datos en entreno en:
        datos de entreno y 
     datos de validación </b>
</div>


In [ ]:
# Dividimos los datos en entreno y validación 

val_size = 0.35  # porcentaje de los datos de entrada que utilizaré para validar el modelo

X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_train_upsampled_scaled, y_train_upsampled_sorted, test_size=val_size,
                                                    shuffle=False)

In [ ]:
# y train tiene observaciones de las dos clases?
y_train.value_counts()


In [ ]:
# y val tiene observaciones de las dos clases?
y_val.value_counts()

In [ ]:
print('Tamañano de los datos de ENTRENO:', X_train_scaled.shape)
print('Tamañano de los datos de TEST:', X_test_scaled.shape)
print('Tamañano de los datos de VALIDACIÓN (cálculo de hiperparámetros):', X_val_scaled.shape)

## *6. Construcción y evaluación de modelos*

* Seleccionamos **[balanced_accuracy]** como métrica de evaluación. 
* Métricas de evaluación disponibles en [Scikit-Learn].


[Scikit-Learn]: https://scikit-learn.org/stable/modules/model_evaluation.html

[balanced_accuracy]: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.balanced_accuracy_score.html

* Recordar utilizar siempre el mismo random_state para poder comparar resultados. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

num_folds = 4
error_metrics = {'balanced_accuracy', 'f1_weighted'}
models = { ('LR', LogisticRegression(solver='saga')),
          ('KNN', KNeighborsClassifier()),
           ('RF', RandomForestClassifier())
         }

results = [] # guarda los resultados de las métricas de evaluación
names = []  # Nombre de cada algoritmo
msg = []  # imprime el resumen del método de cross-validation


<div class="alert alert-success">
    <b> ¿Cuál obtiene mejores resultados? ¿Qué balanced_accuracy obtiene?</b>
</div>


In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit

ite=0

# Entreno con validación cruzada
for scoring in error_metrics:
    print('Métrica de evaluación: ', scoring)
    for name, model in models:
        print('Modelo ', name)
        cross_validation = TimeSeriesSplit(n_splits=num_folds)
        cv_results = cross_val_score(model, X_train_scaled, y_train, cv=cross_validation, scoring=scoring)
        results.append(cv_results)
        if ite == 0:
            names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    print(msg)

    # Comparar resultados entre algoritmos
    fig = plt.figure()
    fig.suptitle('Comparación de algoritmos con métrica de evaluación: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Modelos candidatos')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

    results = []
    
    ite += 1

## *7. Ajustar hiperparámetros*

Pasos para realizar el hiperajuste de los parámetros:
[RandomForest Classifier] parámeteros

* Métrica para optimizar: *balanced_accuracy*
* Definir los rangos de los parámetros de búsqueda: *params*
* Entrenar con los datos de validación: *X_val*

[RandomForest Classifier]:https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [ ]:

from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

# RF
modelo = RandomForestClassifier()
params = {
     'n_estimators': [100, 700], #default=100
    'criterion': ['gini'], #default=gini
     'max_depth': [None],  #default=None
    'class_weight': [None, 'balanced'] # default=None
 }
scoring='balanced_accuracy'
cross_validation = TimeSeriesSplit(n_splits=5)
my_cv = cross_validation.split(X_train_scaled, y_train)

gsearch = GridSearchCV(estimator=modelo, param_grid=params, scoring=scoring, cv=my_cv, verbose=2)
gsearch.fit(X_train_scaled, y_train)

print("Mejor resultado: %f utilizando los siguientes hiperparámetros %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']


In [ ]:
# Best model
best_model = gsearch.best_estimator_
best_model

## *8. Evaluación final del modelo*



Métricas de evaluación:
  * 1. Matriz de confusión
  * 2. Coeficiente de Matthews (MCC)

  
  <div class="alert alert-success">
    <b> Entrena el modelo con los hiperparámetros óptimos encontrados en el apartado anterior y realiza las predicciones. </b>
</div>


In [ ]:

modelo_rf = RandomForestClassifier(n_estimators=700, criterion='gini', max_depth=None, class_weight='balanced')
modelo_rf.fit(X_train_scaled, y_train)  # Se entrena al modelo RF
y_predict = modelo_rf.predict(X_test_scaled)  # Se calculan las predicciones


<div class="alert alert-success">
    <b> Matriz de Confusión </b>
</div>


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

confusion_matrix = confusion_matrix(y_test, y_predict)
print(classification_report(y_test, y_predict))

In [ ]:
sns.heatmap(confusion_matrix, annot=True, fmt='g', cmap="Blues")
# Set the print options to suppress scientific notation
plt.title("Matriz de Confusión")
plt.xlabel("Etiquetas Predichas")
plt.ylabel("Etiquetas Verdaderas")
plt.show()

<div class="alert alert-success">
    <b> Coeficiente de Mathews </b>
</div>


El MCC utiliza coeficientes de correlación entre -1 y +1. 
* Coeficiente +1 representa una predicción perfecta
* Coeficiente 0 representa una predicción media aleatoria
* Coeficiente -1 representa una predicción inversa. 

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_corrcoef(y_test, y_predict)

<div class="alert alert-success">
    <b> Importancia de atributos </b>
</div>



In [ ]:
# Imprimir la importancia de cada atributo (Solo si Random forest es seleccionado)
importancia_atributos = modelo_rf.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importancia_atributos)[::-1]
std = np.std([tree.feature_importances_ for tree in modelo_rf.estimators_], axis=0)

# Print the feature ranking
print("Ranking de importancia de atributos:")
for f in range(X_train_scaled.shape[1]):
    print("%d. Atributo %d (%f)" % (f + 1, indices[f], importancia_atributos[indices[f]]))


In [ ]:

# Grafica la importancia de los atributos
feature_names = X_train_scaled.columns  # creo una lista con el nombre de las features
features = [feature_names[i] for i in indices]  
plt.figure()
plt.title("Feature Importances")
plt.bar(range(X_test_scaled.shape[1]), importancia_atributos[indices], yerr=std[indices], align="center")
plt.xticks(range(X_test_scaled.shape[1]), features, rotation=90)
plt.xlim([-1, X_test_scaled.shape[1]])
plt.show()


<div class="alert alert-success">
    <b> ROC  </b>
</div>


Una técnica ampliamente adoptada para tratar conjuntos de datos muy desequilibrados se denomina remuestreo. Consiste en eliminar muestras de la clase mayoritaria (submuestreo) y/o añadir más ejemplos de la clase minoritaria (sobremuestreo).

<img src="https://raw.githubusercontent.com/sbarja/smart-energy-22-23/main/Figures/roc-auc.png" alt="Drawing" style="width: 800px;"/>



In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# False positive rate, True positive rate
fpr, tpr, thresholds = roc_curve(y_test, y_predict)
auc = roc_auc_score(y_test, y_predict)

plt.plot(fpr, tpr, label='ROC curve (AUC = {:.2f})'.format(auc))
plt.plot([0, 1], [0, 1], 'r--', label='Random Classifier')
plt.title('ROC Curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.legend()
plt.show()